In [1]:
import numpy as np
import math
import random
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import multiprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
import tensorflow as tf
import sklearn
import plotly.offline as pyo

In [2]:
df = pd.read_csv('Datasets/diabetes.csv')
X_df = df[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']]
Y_df = df[['Outcome']]

x_train, x_dev, y_train, y_dev = train_test_split(X_df, Y_df, test_size=0.2, random_state=42)
mu = np.mean(x_train.values, axis=0, keepdims=True)
sigma = np.mean(np.square(x_train.values - mu), axis=0, keepdims=True)
x_train = (x_train - mu) / np.sqrt(sigma)
x_dev = (x_dev - mu) / np.sqrt(sigma)
x_train = tf.cast(x_train.values, tf.float32)
y_train = tf.cast(y_train.values, tf.float32)
x_dev = tf.cast(x_dev.values, tf.float32)
y_dev = tf.cast(y_dev.values, tf.float32)

In [99]:
#ANN
Opt = tf.keras.optimizers.Adam(learning_rate=0.001, weight_decay=0)
model = tf.keras.Sequential([tf.keras.layers.Dense(5, activation="relu"), tf.keras.layers.Dense(1, activation='sigmoid')])
model.compile(optimizer=Opt, loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), metrics=('acc'))
history = model.fit(x_train, y_train, epochs=1000, validation_data=(x_dev, y_dev))
model.evaluate(x_dev, y_dev, verbose=1)

Epoch 1/1000
20/20 [==============================] - 1s 11ms/step - loss: 0.7175 - acc: 0.6336 - val_loss: 0.6203 - val_acc: 0.7143
Epoch 2/1000
20/20 [==============================] - 0s 5ms/step - loss: 0.6974 - acc: 0.6466 - val_loss: 0.6092 - val_acc: 0.7143
Epoch 3/1000
20/20 [==============================] - 0s 5ms/step - loss: 0.6815 - acc: 0.6580 - val_loss: 0.5983 - val_acc: 0.7208
Epoch 4/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.6664 - acc: 0.6726 - val_loss: 0.5883 - val_acc: 0.7273
Epoch 5/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.6522 - acc: 0.6824 - val_loss: 0.5792 - val_acc: 0.7143
Epoch 6/1000
20/20 [==============================] - 0s 4ms/step - loss: 0.6399 - acc: 0.6840 - val_loss: 0.5718 - val_acc: 0.7273
Epoch 7/1000
20/20 [==============================] - 0s 7ms/step - loss: 0.6287 - acc: 0.6873 - val_loss: 0.5663 - val_acc: 0.7468
Epoch 8/1000
20/20 [==============================] - 0s 9ms/step - loss: 0

5/5 [==============================] - 0s 1ms/step - loss: 0.5723 - acc: 0.7208


[0.5723373293876648, 0.7207792401313782]

In [100]:
# Plot the training history
fig = go.Figure()
fig.add_trace(go.Scatter(y=history.history['loss'], mode='lines', name='Train Loss'))
fig.add_trace(go.Scatter(y=history.history['val_loss'], mode='lines', name='Validation Loss'))
fig.update_layout(title='Training Loss', xaxis_title='Epoch', yaxis_title='Loss')
fig.show()
pyo.plot(fig, filename='TrainingLoss.html', auto_open=False)

fig = go.Figure()
fig.add_trace(go.Scatter(y=history.history['acc'], mode='lines', name='Train Accuracy'))
fig.add_trace(go.Scatter(y=history.history['val_acc'], mode='lines', name='Validation Accuracy'))
fig.update_layout(title='Training Accuracy', xaxis_title='Epoch', yaxis_title='Accuracy')
fig.show()
pyo.plot(fig, filename='TrainingAccuracy.html', auto_open=False)

'TrainingAccuracy.html'

In [162]:
#RF
clf = RandomForestClassifier(n_estimators=10000)
clf.fit(x_train, y_train)
np.count_nonzero(np.int64(clf.predict(x_dev).reshape((-1, 1)) == y_dev.numpy().reshape((-1, 1))))/(y_dev.shape[0])

C:\Users\Admin\anaconda3\lib\site-packages\sklearn\base.py:1151: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



0.7402597402597403

In [159]:
y_predict = clf.predict_proba(x_train).reshape((-1, 2))[:, 1].reshape(-1, 1)
display(-np.mean((y_train.numpy() * np.log(np.clip(y_predict, 0.001, 1))) + (
                            (1 - y_train.numpy()) * np.log(1 - np.clip(y_predict, 0, 0.999))), axis=0, keepdims=True))
display(np.count_nonzero(np.int64(clf.predict(x_dev).reshape((-1, 1)) == y_dev.numpy().reshape((-1, 1))))/(y_dev.shape[0]))

array([[0.12948401]])

0.7467532467532467

In [153]:
clf.predict_proba(x_dev).reshape((-1, 2))[:, 1]
# np.hstack((clf.predict(x_train).reshape((-1,1)), y_train))

array([0.4621, 0.2228, 0.1223, 0.2189, 0.4749, 0.6073, 0.0093, 0.6785,
       0.5927, 0.6415, 0.2338, 0.7997, 0.2725, 0.4498, 0.0248, 0.4443,
       0.1067, 0.0388, 0.5458, 0.4853, 0.3792, 0.0859, 0.285 , 0.0325,
       0.5897, 0.8753, 0.081 , 0.0225, 0.1356, 0.1566, 0.7234, 0.7108,
       0.7524, 0.8031, 0.5576, 0.7728, 0.8163, 0.5085, 0.1763, 0.7318,
       0.0659, 0.3277, 0.7103, 0.4246, 0.0624, 0.635 , 0.5304, 0.2159,
       0.1645, 0.8654, 0.0179, 0.8164, 0.8006, 0.2464, 0.0861, 0.0592,
       0.5624, 0.0247, 0.2689, 0.7776, 0.6833, 0.218 , 0.3556, 0.4132,
       0.0798, 0.7226, 0.0139, 0.5669, 0.0349, 0.723 , 0.7366, 0.1429,
       0.112 , 0.0448, 0.1294, 0.469 , 0.2507, 0.1261, 0.149 , 0.2187,
       0.71  , 0.1371, 0.1672, 0.5526, 0.3165, 0.892 , 0.6434, 0.4393,
       0.2437, 0.0539, 0.0229, 0.2102, 0.0163, 0.5301, 0.4376, 0.5852,
       0.6644, 0.0763, 0.7238, 0.0898, 0.7894, 0.0532, 0.4889, 0.5831,
       0.8104, 0.2121, 0.2398, 0.8994, 0.2471, 0.7661, 0.0366, 0.5164,
      

In [141]:
#Log Regression
Opt = tf.keras.optimizers.Adam(learning_rate=0.0005, weight_decay=0)
modelLR = tf.keras.Sequential([tf.keras.layers.Dense(1, activation='sigmoid')])
modelLR.compile(optimizer=Opt, loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), metrics=('acc'))
modelLR.fit(x_train, y_train, epochs=1000)
modelLR.evaluate(x_dev, y_dev, verbose=1)

Epoch 1/1000
20/20 [==============================] - 1s 2ms/step - loss: 0.9111 - acc: 0.4853
Epoch 2/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.9005 - acc: 0.4870
Epoch 3/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.8906 - acc: 0.4935
Epoch 4/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.8812 - acc: 0.5000
Epoch 5/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.8722 - acc: 0.5081
Epoch 6/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.8629 - acc: 0.5163
Epoch 7/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.8540 - acc: 0.5195
Epoch 8/1000
20/20 [==============================] - 0s 3ms/step - loss: 0.8452 - acc: 0.5277
Epoch 9/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.8365 - acc: 0.5309
Epoch 10/1000
20/20 [==============================] - 0s 2ms/step - loss: 0.8281 - acc: 0.5391
Epoch 11/1000
20/20 [============================

[0.5097157955169678, 0.7532467246055603]

In [163]:
performanceComp = pd.DataFrame({'Algorithm': ['Neural Network', 'Random Forest', 'Logistic Regression'], 'Training Loss': [0.4145, 0.1295, 0.4679], 'Training Accuracy': [0.8062, 1.0000, 0.7704], 'Validation Loss': [0.5723, 0.4903, 0.5097], 'Validation Accuracy': [0.7207, 0.7467, 0.7532]})
performanceComp

,Algorithm,Training Loss,Training Accuracy,Validation Loss,Validation Accuracy
0,Neural Network,0.4145,0.8062,0.5723,0.7207
1,Random Forest,0.1295,1.0000,0.4903,0.7467
2,Logistic Regression,0.4679,0.7704,0.5097,0.7532
